In [1]:
import cv2
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
np.random.seed(72)

In [3]:
def read_image(path="ORL3232", category=10):
    def one_hot(x):
        z = np.zeros(category)
        z[x - 1] = 1
        return z

    bmps = Path(path).rglob("*.bmp")
    data = {"labels": [], "ids": [], "images": []}

    for bmp in bmps:
        if not bmp.parent.stem.isdigit():
            continue

        if not int(bmp.parent.stem) <= 10:
            continue

        data["labels"].append(int(bmp.parent.stem))
        data["ids"].append(int(bmp.stem))
        data["images"].append(cv2.imread(str(bmp))[:, :, 0].squeeze().flatten())

    dataframe = pd.DataFrame(data).sample(frac=1)
    dataframe['labels_onehot'] = dataframe['labels'].apply(lambda x: one_hot(x))

    train = dataframe[dataframe['ids'] % 2 == 1]
    test = dataframe[dataframe['ids'] % 2 == 0]

    images = np.stack(train['images'].tolist() + test['images'].tolist())
    labels = np.stack(train['labels'].tolist() + test['labels'].tolist())
    labels_onehot = np.stack(train['labels_onehot'].tolist() + test['labels_onehot'].tolist())

    return images, labels, labels_onehot

In [4]:
def pca(data, n_components):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    data_normalized = (data - mean) / std
    cov_matrix = np.cov(data_normalized, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
    sorted_index = np.argsort(eigenvalues)[::-1]
    sorted_eigenvectors = eigenvectors[:, sorted_index]
    feature_vectors = sorted_eigenvectors[:, :n_components]
    pca_data = np.dot(data_normalized, feature_vectors)
    return pca_data


def lda(X, y, num_components=2):
    class_labels = np.unique(y)
    mean_overall = np.mean(X, axis=0)
    S_W = np.zeros((X.shape[1], X.shape[1]))
    S_B = np.zeros((X.shape[1], X.shape[1]))
    for c in class_labels:
        X_c = X[y == c]
        mean_c = np.mean(X_c, axis=0)
        S_W += np.dot((X_c - mean_c).T, (X_c - mean_c))
        n_c = X_c.shape[0]
        mean_diff = (mean_c - mean_overall).reshape(-1, 1)
        S_B += n_c * np.dot(mean_diff, mean_diff.T)
    A = np.dot(np.linalg.inv(S_W), S_B)
    eigenvalues, eigenvectors = np.linalg.eig(A)
    eigenvectors = eigenvectors[:, np.argsort(eigenvalues)[::-1]]
    W = eigenvectors[:, :num_components]
    return np.real(W)


def euclidean_similarity(A, B):
    A = A[:, np.newaxis, :]
    B = B[np.newaxis, :, :]
    distances = np.sqrt(np.sum((A - B) ** 2, axis=2))
    similarity = 1 / (1 + distances)
    return similarity

In [5]:
def normalize(data):
    data = data / 255
    return data


def min_max_normalize(data):
    data = (data - np.min(data)) / (np.max(data) - np.min(data))
    return data

In [6]:
images, labels, labels_onehot = read_image()
images = min_max_normalize(images)
convert_matrix = lda(images, labels, 2)
images = np.dot(images, convert_matrix)

In [7]:
train_images, test_images = images[:50, :], images[50:, :]
train_labels_onehot, test_labels_onehot = labels_onehot[:50, :], labels_onehot[50:, :]
train_labels, test_labels = labels[:200], labels[200:]

In [8]:
def levenberg_marquardt(cost_func, x0, args=(), max_iter=100, lambda_=0.01, up_factor=10, down_factor=9, tol=1e-6):
    x = x0.copy()
    for i in range(max_iter):
        residuals = cost_func(x, *args)
        cost = 0.5 * np.sum(residuals**2)

        # 计算雅可比矩阵
        J = approx_jacobian(x, cost_func, residuals, *args)

        # 计算梯度
        gradient = J.T @ residuals

        # 计算 Hessian 矩阵的近似
        A = J.T @ J

        # 调整阻尼因子
        eye = np.eye(len(x))
        while True:
            # 解更新方程（带阻尼的高斯-牛顿方程）
            delta = np.linalg.solve(A + lambda_ * eye, -gradient)
            new_x = x + delta
            new_residuals = cost_func(new_x, *args)
            new_cost = 0.5 * np.sum(new_residuals**2)

            # 检查是否改善
            if new_cost < cost:
                # 成功，减小阻尼因子
                lambda_ /= down_factor
                x = new_x
                if np.linalg.norm(delta) < tol:
                    break
                break
            else:
                # 失败，增大阻尼因子
                lambda_ *= up_factor

    return x

def approx_jacobian(x, func, f0, *args, eps=1e-8):
    jacobian = np.zeros((len(f0), len(x)))
    for i in range(len(x)):
        dx = np.zeros_like(x)
        dx[i] = eps
        fi = func(x + dx, *args)
        jacobian[:, i] = (fi - f0) / eps
    return jacobian


In [9]:
import numpy as np

def sigmoid(x):
    x_clipped = np.clip(x, -20, 20)
    return np.where(x_clipped >= 0, 1 / (1 + np.exp(-x_clipped)), np.exp(x_clipped) / (1 + np.exp(x_clipped)))


def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))


input_size = 2  
hidden_size = 8  
output_size = 10  

def cost_function(weights, X, y):
    split = hidden_size * (input_size + 1)
    hidden_weights = weights[:split].reshape((hidden_size, input_size + 1))
    output_weights = weights[split:].reshape((output_size, hidden_size + 1))

    hidden_layer_input = np.dot(X, hidden_weights[:, :-1].T) + hidden_weights[:, -1]
    hidden_layer_output = sigmoid(hidden_layer_input)

    output_layer_input = np.dot(hidden_layer_output, output_weights[:, :-1].T) + output_weights[:, -1]
    output_layer_output = sigmoid(output_layer_input)

    residuals = output_layer_output.ravel() - y.ravel()
    return residuals.ravel()


def predict(weights, X):
    split = hidden_size * (input_size + 1)
    hidden_weights = weights[:split].reshape((hidden_size, input_size + 1))
    output_weights = weights[split:].reshape((output_size, hidden_size + 1))

    hidden_layer_input = np.dot(X, hidden_weights[:, :-1].T) + hidden_weights[:, -1]
    hidden_layer_output = sigmoid(hidden_layer_input)

    output_layer_input = np.dot(hidden_layer_output, output_weights[:, :-1].T) + output_weights[:, -1]
    output_layer_output = sigmoid(output_layer_input)
    return output_layer_output

initial_weights = np.random.randn(hidden_size * (input_size + 1) + output_size * (hidden_size + 1))
result = levenberg_marquardt(cost_function, initial_weights, args=(train_images, train_labels_onehot))
result

array([-6.04481242e+00, -1.74213171e+00,  2.06429148e+00,  4.01488121e+01,
        1.10103336e+01, -5.27232162e+00, -7.57947734e+00, -2.97430273e+01,
       -2.67951756e-02,  2.72129486e+01,  9.76645817e+00, -2.40937854e+00,
       -2.67312310e+01,  1.47694032e+01,  2.60488236e+00,  8.85964782e+00,
        1.35373950e+01, -1.02262029e+00, -2.71178517e+01,  3.02802421e+01,
        5.47891071e+00, -2.03087941e+01, -1.45164102e+01,  2.18269281e+00,
        1.10893160e+00, -2.26913025e+01, -1.18746253e+01, -1.50313236e+01,
        1.72920135e+01, -4.08898340e+00,  1.67759590e+01,  4.32274768e+00,
       -6.22829471e+00, -7.19624563e+00,  2.94398490e+01, -8.83290588e+00,
        1.81886237e+01, -2.05333114e+01,  4.59874629e+00, -1.60188272e+01,
       -1.59860016e+01, -3.67745440e+00,  2.33939507e+00, -1.81261314e+01,
        2.43297253e+01, -1.35161137e+01, -6.22839634e+00, -1.22937789e+01,
       -1.52309135e+01,  1.65263873e+01,  5.50195833e+00,  1.64789077e-01,
       -5.57112680e+00,  

In [10]:
pred = predict(result, test_images)
np.average(np.argmax(pred, axis=1) == np.argmax(test_labels_onehot, axis=1))

0.9